# Bike Thefts Berlin - EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
PATH = '../data'

In [3]:
time_parseable = ['start_date_delict', 'end_date_delict']

## Prep data inspection

In [4]:
class BikeThefts():
    '''Read, inspect and transform initial data.'''

    def __init__(self, path = PATH) -> None:
        self.path = path

    def read_initial_data(self, file) -> pd.DataFrame:
        '''Return DataFrame with feature matrix and labels as values.'''
        df = pd.read_csv(f'{self.path}/{file}', index_col=0, parse_dates=True, encoding = 'unicode_escape')
        return df

    def read_extracted_data(self, file) -> pd.DataFrame:
        '''Return DataFrame extracted from initial data.'''
        df = pd.read_csv(f'{self.path}/{file}', encoding = 'unicode_escape')
        return df
    
    def check_unique(self, serie) -> pd.Series:
        '''Return unique values of selected columns.'''
        return serie.unique()

    def include_timestamps(self, df) -> pd.DataFrame:
        '''Return DataFrame with time-stamps.'''
        df['year'] = df.index.year
        df['month'] = df.index.month
        return df

    def time_parser(self, df, time_parsables) -> pd.DataFrame:
        '''Parse columns encoded as strings to datetime-objects.'''
        for col in time_parsables:
            df[col] = pd.to_datetime(df[col])
        return df

    def crime_duration_days(self, df, start, end) -> pd.DataFrame: #use for plotting?
        '''Calculate duration of crime in days.'''
        df['crime_duration_days'] = end - start
        return df
    
    def crime_duration_hours(self, df, start, end) -> pd.DataFrame: #use for plotting?
        '''Calculate duration of crime in hours.'''
        df['crime_duration_hours'] = abs(start - end)
        return df
    
    def fill_ints(self, df) -> pd.DataFrame:
        '''Reencode LOR into 8-digit values.'''
        df['LOR'] = df['LOR'].apply(lambda x: str(x))
        df['LOR'] = df['LOR'].apply(lambda x: f'0{(x)}'[-8:])
        return df
    
    def fill_ints_grouped(self, df) -> pd.DataFrame:
        '''Group by LOR. Return df including number of bike thefts per LOR-group.'''
        df_grouped = pd.DataFrame(df.groupby('LOR').size(),
                          columns=['bike_thefts']).reset_index()
        return df_grouped
        
    def rename_cols(self, df) -> pd.DataFrame:
        return df.rename(columns={ 
            'ANGELEGT_AM' : 'track_date',
            'TATZEIT_ANFANG_DATUM' : 'start_date_delict',
            'TATZEIT_ANFANG_STUNDE' : 'start_time_delict',
            'TATZEIT_ENDE_DATUM' : 'end_date_delict',
            'TATZEIT_ENDE_STUNDE' : 'end_time_delict',
            'SCHADENSHOEHE' : 'damage_amount',
            'VERSUCH' : 'intent_delict',
            'ART_DES_FAHRRADS' : 'bike_type',
            'DELIKT' : 'delict',
            'ERFASSUNGSGRUND' : 'description'
            })
    
    def save_intermediate_data(self, df, file: str):
        '''Save extracted data locally as csv-file.'''
        return df.to_csv(f'{self.path}/{file}.csv')

    def save_LOR_bike_thefts(self, df, group_by: str, col_names: str, file: str):
        '''Save extracted LOR-bike thefts-data locally as csv-file.'''
        bike_thefts_LOR = pd.DataFrame(df.groupby(group_by).size(),
                        columns = [col_names]).reset_index()
        return bike_thefts_LOR.to_csv(f'{self.path}/{file}.csv')

In [5]:
# class PlotBikeThefts():
#     '''Plot data with seaborn.'''

#     def plot_categoricals(self, df, ordinate) -> sns:
#             return sns.catplot(
#             data=df, y=ordinate, kind="count",
#             palette="pastel", edgecolor=".6",
#         )

#     def plot_correlations(self, df) -> sns:
#         corr = df.corr()
#         mask = np.triu(np.ones_like(corr, dtype=bool))
#         f, ax = plt.subplots(figsize=(8, 6))
#         plt.xticks(rotation=45)
#         cmap = sns.diverging_palette(230, 20, as_cmap=True)
#         return sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
#                     square=True, linewidths=.5, cbar_kws={"shrink": .5})
    

## Create transformed dataset

In [6]:
bike_thefts = BikeThefts()

In [7]:
df = bike_thefts.read_initial_data('Fahrraddiebstahl.csv')
df.head()

,TATZEIT_ANFANG_DATUM,TATZEIT_ANFANG_STUNDE,TATZEIT_ENDE_DATUM,TATZEIT_ENDE_STUNDE,LOR,SCHADENSHOEHE,VERSUCH,ART_DES_FAHRRADS,DELIKT,ERFASSUNGSGRUND
ANGELEGT_AM,,,,,,,,,,
2023-02-19,18.02.2023,18,19.02.2023,7,3701658,3472,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
2023-02-19,18.02.2023,20,19.02.2023,9,1100308,2277,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
2023-02-19,18.02.2023,16,19.02.2023,0,6100102,746,Nein,Damenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
2023-02-19,18.02.2023,17,19.02.2023,0,7601544,1849,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
2023-02-19,17.02.2023,17,17.02.2023,17,3601346,1863,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern


In [8]:
df = bike_thefts.rename_cols(df)
df.columns

Index(['start_date_delict', 'start_time_delict', 'end_date_delict',
       'end_time_delict', 'LOR', 'damage_amount', 'intent_delict', 'bike_type',
       'delict', 'description'],
      dtype='object')

## Parse time

In [9]:
df_transformed = bike_thefts.time_parser(df, time_parseable)
df_transformed.head(3)

,start_date_delict,start_time_delict,end_date_delict,end_time_delict,LOR,damage_amount,intent_delict,bike_type,delict,description
ANGELEGT_AM,,,,,,,,,,
2023-02-19,2023-02-18,18,2023-02-19,7,3701658,3472,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
2023-02-19,2023-02-18,20,2023-02-19,9,1100308,2277,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
2023-02-19,2023-02-18,16,2023-02-19,0,6100102,746,Nein,Damenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern


In [10]:
df_transformed = bike_thefts.crime_duration_days(df_transformed, 
                                            start=df_transformed['start_date_delict'], 
                                            end = df_transformed['end_date_delict'])

In [12]:
df_transformed = bike_thefts.crime_duration_hours(df_transformed, 
                                            start=df_transformed['start_time_delict'], 
                                            end = df_transformed['end_time_delict'])
df_transformed.columns

Index(['start_date_delict', 'start_time_delict', 'end_date_delict',
       'end_time_delict', 'LOR', 'damage_amount', 'intent_delict', 'bike_type',
       'delict', 'description', 'crime_duration_days', 'crime_duration_hours'],
      dtype='object')

In [14]:
df_transformed = bike_thefts.include_timestamps(df_transformed)
df_transformed.columns

Index(['start_date_delict', 'start_time_delict', 'end_date_delict',
       'end_time_delict', 'LOR', 'damage_amount', 'intent_delict', 'bike_type',
       'delict', 'description', 'crime_duration_days', 'crime_duration_hours',
       'year', 'month'],
      dtype='object')

In [16]:
df_transformed = bike_thefts.fill_ints(df_transformed)
df_transformed.columns

Index(['start_date_delict', 'start_time_delict', 'end_date_delict',
       'end_time_delict', 'LOR', 'damage_amount', 'intent_delict', 'bike_type',
       'delict', 'description', 'crime_duration_days', 'crime_duration_hours',
       'year', 'month'],
      dtype='object')

In [17]:
bike_thefts.save_intermediate_data(df_transformed, 'bike_thefts_transformed') # assign additional col with district names

## Create sub DataFrame with `LOR` and `bike_theft_count`

In [ ]:
df_count_bike_thefts = bike_thefts.fill_ints_grouped(df_transformed)
df_count_bike_thefts.head(2)
#len(df_count_bike_thefts)

In [ ]:
len(df_transformed)

In [ ]:
bike_thefts.save_LOR_bike_thefts(df_count_bike_thefts, 'LOR', 'bike_thefts_count', 'bike_thefts_LOR' )

## Extract dataframes for individual years

In [ ]:
extract_2022 = df_transformed.loc['2022-01-01':'2022-12-31']
extract_2022.head(2)

In [ ]:
extract_22_resampled = extract_2022.resample('M')
extract_22_resampled = extract_22_resampled['LOR', 'damage_amount'].mean().round(2)
bike_thefts.save_intermediate_data(extract_22_resampled, 'extract_22_resampled')

## Extract series for individual years and plot numeric data: "Schadenshöhe"

In [ ]:
extract_22_resampled.plot.line(title = 'Mean Values of Monthly Bike Thefts (EUR) - 2022'); #how can this be rendered in app?

In [ ]:
extract_22_resampled.plot.hist(bins = 10, title = 'Values of Monthly Bike Thefts - 2022'); #how can this be rendered in app?

In [ ]:
extract_2023 = df_transformed.loc['2023-01-01':'2023-02-19']
extract_2023.head(2)

In [ ]:
extract_23_resampled = extract_2023.resample('M')
extract_23_resampled = extract_23_resampled['LOR', 'damage_amount'].mean().round(2)
bike_thefts.save_intermediate_data(extract_23_resampled, 'extract_23_resampled')

In [ ]:
bike_thefts.save_intermediate_data(extract_2022, 'bike_thefts_df_2022')
bike_thefts.save_intermediate_data(extract_2023, 'bike_thefts_df_2023')

## Extract series for individual years and plot numeric data: thefts

In [ ]:
extract_2022.reset_index(inplace=True)
extract_2022.head(2)

In [ ]:
bike_thefts_series_22 = pd.DataFrame(extract_2022.groupby(['LOR', 'ANGELEGT_AM']).size(),
                       columns = ['bike_theft_count']).reset_index()
bike_thefts_series_22 = bike_thefts_series_22.set_index(bike_thefts_series_22['ANGELEGT_AM'])
bike_thefts_series_22 = bike_thefts_series_22['bike_theft_count']
bike_thefts_series_22.hist();

In [ ]:
bike_thefts_series_22.index

In [ ]:
bike_thefts_series_22.describe() #render in app

In [ ]:
(bike_thefts_series_22
.resample('D')  #change frequency
.mean()
.plot.line()
);

In [ ]:
bike_thefts_series_22.isna().sum().sum()

In [ ]:
(bike_thefts_series_22
 .resample('M')
 .sum()
 .div(bike_thefts_series_22.sum())
 .mul(100)
 .rename(lambda idx: idx.month_name())
 .plot.barh(title = '2022 Monthly Percent Bike Thefts')
 );

In [ ]:
bike_thefts.save_intermediate_data(bike_thefts_series_22, 'bike_thefts_series_2022')

In [ ]:
extract_2023.reset_index(inplace=True)
extract_2023.head(2)

In [ ]:
bike_thefts_series_23 = pd.DataFrame(extract_2023.groupby(['LOR', 'ANGELEGT_AM']).size(),
                       columns = ['bike_theft_count']).reset_index()
bike_thefts_series_23 = bike_thefts_series_23.set_index(bike_thefts_series_23['ANGELEGT_AM'])
bike_thefts_series_23 = bike_thefts_series_23['bike_theft_count']
bike_thefts_series_23.hist();

In [ ]:
bike_thefts_series_23.describe()

In [ ]:
(bike_thefts_series_23
.resample('D')
.mean()
.plot.line()
);

In [ ]:
(bike_thefts_series_23
.resample('W')
.mean()
.plot.barh()
);

In [ ]:
(bike_thefts_series_23
 .resample('M')
 .sum()
 .div(bike_thefts_series_23.sum())
 .mul(100)
 .rename(lambda idx: idx.month_name())
 .plot.barh(title = '2023 Monthly Percent Bike Thefts')
 );

In [ ]:
bike_thefts.save_intermediate_data(bike_thefts_series_23, 'bike_thefts_series_2023')

In [ ]:
df_transformed.drop(df_transformed.iloc[:, 0:4], axis=1, inplace=True)
df_transformed.head(2) #check

In [ ]:
df_transformed.reset_index(inplace=True)
df_transformed.head(2)

In [ ]:
bike_thefts_LOR_year = pd.DataFrame(df_transformed.groupby(['LOR', 'year']).size(),
                       columns = ['bike_theft_count'])
bike_thefts_LOR_year = bike_thefts_LOR_year.reset_index()
bike_thefts_LOR_year = bike_thefts_LOR_year.set_index('year').reset_index()
bike_thefts_LOR_year.head(2)

In [ ]:
bike_thefts.save_LOR_bike_thefts(
    df = bike_thefts_LOR_year, 
    group_by =['LOR', 'year'],
    col_names='bike_theft_count',
    file = 'bike_thefts_year_LOR_count'
    ) #check